Hybrid: Kết hợp nhiều thuật toán 
- Cùng loại đầu vào: 
    - Pipeline: xử lý data --> rẽ nhánh cho từng pp riêng.
- Các pp đưa data vào khác nhau. Rẽ nhánh ngay từ đầu.

Ensemble: Cùng 1 pipiline, là 1 thuật toán
- Bên trong thuật toán đã kết hợp nhiều các xử lý tối ưu 

- Accuracy vs performance trade-off

In [3]:
from sklearn.ensemble import RandomForestClassifier

Database Xoa DB --> Read only --> Database mới

Request --> Collect. 



pipeline: xu ly tat ca data. 

-Performance: < 0.5s

Ensemble: 
    
    - Bagging (Boostrap Aggregation): Random Forest (Scikit-learn)
    - Boosting: AdaBoost, Gradient Boosting (Scitkit, XGBoost, CatBoost, LightGBM)
    - Stacking (Sklearn, Stacking...).

KNN: K-nearest neighbor --> lazy learning

Aggregration function: Average, median, count, min, max.

In [83]:
from sklearn.datasets import load_iris, load_diabetes, load_breast_cancer
from sklearn.ensemble import StackingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier

In [19]:
data = load_iris()

In [20]:
data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [21]:
df = pd.DataFrame(data.data, columns=data.feature_names)
df['species'] = data.target
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [23]:
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [26]:

df['species_name'] = df.species.map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})

In [27]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species,species_name
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa


# Stacking

In [52]:
estimators = [('rf', KNeighborsClassifier(n_neighbors=5)), 
              ('svr', make_pipeline(StandardScaler(), LinearSVC(random_state=42)))]

clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

In [53]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :4], df.iloc[:, 4], test_size=0.2, shuffle=True, 
                                                    random_state=4)

In [54]:
(X_train.shape, y_train.shape)

((120, 4), (120,))

In [55]:
clf.fit(X_train, y_train)

StackingClassifier(estimators=[('rf', KNeighborsClassifier()),
                               ('svr',
                                Pipeline(steps=[('standardscaler',
                                                 StandardScaler()),
                                                ('linearsvc',
                                                 LinearSVC(random_state=42))]))],
                   final_estimator=LogisticRegression())

In [56]:
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      0.80      0.89         5
           2       0.90      1.00      0.95         9

    accuracy                           0.97        30
   macro avg       0.97      0.93      0.95        30
weighted avg       0.97      0.97      0.97        30



In [57]:
clf.score(X_test, y_test)

0.9666666666666667

In [45]:
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      0.80      0.89         5
           2       0.90      1.00      0.95         9

    accuracy                           0.97        30
   macro avg       0.97      0.93      0.95        30
weighted avg       0.97      0.97      0.97        30



In [47]:
clf.predict(X_test)

array([2, 0, 2, 2, 2, 1, 2, 0, 0, 2, 0, 0, 0, 1, 2, 0, 1, 0, 0, 2, 0, 2,
       1, 0, 0, 0, 0, 0, 0, 2])

In [49]:
clf.score(X_test, y_test)

0.9666666666666667

# Bagging = Boostrap + Aggregation
Boostrap sampling: sampling with replacement
-Random Forest
+ Column sampling
+ Row sampling
+ Aggregation

In [62]:
rf = RandomForestClassifier(n_estimators=50, max_depth=3, min_samples_leaf=5)
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=3, min_samples_leaf=5, n_estimators=50)

In [63]:
print(classification_report(y_test, rf.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      0.80      0.89         5
           2       0.90      1.00      0.95         9

    accuracy                           0.97        30
   macro avg       0.97      0.93      0.95        30
weighted avg       0.97      0.97      0.97        30



# Boosting
Gradient Boosting: D1 = (X, y)
 - Fit model decision tree on data --> y_predicted_1
 - Calculate **error_1 = y - y_predicted_1**  --> 
 
 - Establish new dataset D2 = (X, error_1)
 - Fit model on new dataset D2 --> y_predicted_2 (~ error_1 = y_true - y_predicted_1 (y_true))
  
 
 - y_predicted_2 = y_predicted_1 + y_predicted_2 --> (y_true) (y_predicted_2 = y_predicted_1 + y_true - y_predicted_1)
 
 
 - Calculate error_2 = y_true - y_predicted_2
 
 - Establish new dataset D3 = (X, error_2)
 
y_final = 0.1* y_predicted_1 + 0.2* y_predicted_2 + ... + 0.5*y_predicted_n
 

In [77]:
data = load_breast_cancer()

In [78]:
data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [82]:
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, shuffle=True)

In [85]:
X_train.shape, X_test.shape

((455, 30), (114, 30))

In [89]:
gb = GradientBoostingClassifier()
params = {'loss' : ['deviance', 'exponential'],
          'learning_rate': [0.001, 0.01, 0.1],
          'min_samples_leaf': [3, 5, 7],
          'max_depth': [3, 2, 4],
          'max_features': [20, 25, 30]
}

grid = GridSearchCV(gb, params, n_jobs=7, cv=5, return_train_score=True, verbose=1)

In [90]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    8.3s
[Parallel(n_jobs=7)]: Done 186 tasks      | elapsed:   26.2s
[Parallel(n_jobs=7)]: Done 436 tasks      | elapsed:   53.9s
[Parallel(n_jobs=7)]: Done 786 tasks      | elapsed:  1.6min
[Parallel(n_jobs=7)]: Done 810 out of 810 | elapsed:  1.7min finished


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=7,
             param_grid={'learning_rate': [0.001, 0.01, 0.1],
                         'loss': ['deviance', 'exponential'],
                         'max_depth': [3, 2, 4], 'max_features': [20, 25, 30],
                         'min_samples_leaf': [3, 5, 7]},
             return_train_score=True, verbose=1)

In [103]:
threshold = 0.9
y_predicted = np.where(grid.best_estimator_.predict_proba(X_test)[:, 1] > threshold, 1, 0)

print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99        42
           1       0.99      1.00      0.99        72

    accuracy                           0.99       114
   macro avg       0.99      0.99      0.99       114
weighted avg       0.99      0.99      0.99       114



F1 = 2  * (precision * recall) / (precision + recall) -> harmonic mean

In [ ]:
Fbeta